Rachunek prawdopodobieństwa i statystyka 
Piotr Śmiałek 

Będę korzystał z API yfinance, która umożliwi mi pobieranie danych z yahoo
Wiecej informacji
https://pypi.org/project/yfinance/

In [1]:
!pip install yfinance


[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: C:\Users\piotr\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Importowanie niezbędnych bibliotek

In [2]:
import yfinance as yf
import sqlite3
import numpy as np
import pandas as pd
import matplotlib as mplt
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

Importowanie danych

Do importowania danych z bazy użyję funkcji:

In [7]:
def import_data(symbol, database_name, data_intervals):
    data = yf.download(symbol, period="max", interval=data_intervals)
    conn = sqlite3.connect("./data/" + database_name)
    data.to_sql(name=symbol, con=conn , if_exists="replace", index=True)
    conn.close()

Za pomocą powyższej funkcji bedę mógł stworzyć 4 osobne zbiory danych dla firmy Microsoft, dla interwałów:
dniowego, tygodniowego, miesięcznego i kwartalnego.

In [8]:
import_data("MSFT", "microsoft_daily.db", "1d")
import_data("MSFT", "microsoft_weekly.db", "1wk")
import_data("MSFT", "microsoft_monthly.db", "1mo")
import_data("MSFT", "microsoft_quarterly.db", "3mo")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Poniższa funkcja umożliwi mi zapisanie danych z SQL do DataFrame.

In [9]:
def convert_to_dataframe(symbol,database_path):
    conn = sqlite3.connect(database_path)
    query = f"SELECT * FROM {symbol}"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

Konwersja danych do DataFrame

In [12]:
df_microsoft_daily = convert_to_dataframe("MSFT", "./data/microsoft_daily.db")
df_microsoft_weekly = convert_to_dataframe("MSFT", "./data/microsoft_weekly.db")
df_microsoft_monthly = convert_to_dataframe("MSFT", "./data/microsoft_monthly.db")
df_microsoft_quarterly = convert_to_dataframe("MSFT", "./data/microsoft_quarterly.db")

Sprawdzenie poprawności wczytanych danych:

In [14]:
df_microsoft_daily.shape

(9524, 7)

In [15]:
df_microsoft_weekly.shape

(1973, 7)

In [16]:
df_microsoft_monthly.shape

(453, 7)

In [17]:
df_microsoft_quarterly.shape

(152, 7)

Każda z wczytanych tabel posiada 7 kolumn:
- Date

- Open - otwarcie to cena instrumentu finansowego (takiego jak akcje) w momencie otwarcia sesji handlowej na rynku. Oznacza to, ile jednostek waluty trzeba było zapłacić lub otrzymać w zamian za jednostkę instrumentu finansowego napoczątku sesji.

- High - najwyższa cena instrumentu finansowego osiągnięta w trakcie danego okresu czasu (np. jednej sesji handlowej). To jest największa wartość, którą osiągnęła cena w danym przedziale czasowym.

- Low - najniższa cena instrumentu finansowego osiągnięta w trakcie danego okresu czasu. To jest najmniejsza wartość, którą osiągnęła cena w danym przedziale czasowym.

- Close - cena instrumentu finansowego w momencie zamknięcia sesji handlowej. Oznacza to, ile jednostek waluty trzeba było zapłacić lub otrzymać w zamian za jednostkę instrumentu finansowego na końcu sesji.

- Adj Close - cena instrumentu finansowego w momencie zamknięcia sesji handlowej. Oznacza to, ile jednostek waluty trzeba było zapłacić lub otrzymać w zamian za jednostkę instrumentu finansowego na końcu sesji.

- Volume - wolumen to liczba jednostek instrumentu finansowego, które zostały wymienione w danym okresie czasu. Oznacza to, ile razy dany instrument finansowy został kupiony lub sprzedany w danym przedziale czasowym.

In [18]:
df_microsoft_daily.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-13 00:00:00,0.088542,0.101563,0.088542,0.097222,0.060274,1031788800
1,1986-03-14 00:00:00,0.097222,0.102431,0.097222,0.100694,0.062427,308160000
2,1986-03-17 00:00:00,0.100694,0.103299,0.100694,0.102431,0.063504,133171200
3,1986-03-18 00:00:00,0.102431,0.103299,0.098958,0.099826,0.061888,67766400
4,1986-03-19 00:00:00,0.099826,0.100694,0.097222,0.098090,0.060812,47894400


In [19]:
df_microsoft_weekly.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-10 00:00:00,0.088542,0.102431,0.088542,0.100694,0.062427,1339948800
1,1986-03-17 00:00:00,0.100694,0.103299,0.091146,0.092882,0.057583,367257600
2,1986-03-24 00:00:00,0.092882,0.096354,0.089410,0.096354,0.059736,136972800
3,1986-03-31 00:00:00,0.096354,0.098958,0.093750,0.096354,0.059736,100598400
4,1986-04-07 00:00:00,0.096354,0.101563,0.092882,0.099826,0.061889,70070400


In [20]:
df_microsoft_monthly.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-04-01 00:00:00,0.095486,0.121962,0.092882,0.111979,0.069423,547142400
1,1986-05-01 00:00:00,0.111979,0.123264,0.107639,0.121528,0.075343,333446400
2,1986-06-01 00:00:00,0.121528,0.121528,0.102431,0.106771,0.066194,293788800
3,1986-07-01 00:00:00,0.106771,0.109375,0.090278,0.098958,0.061350,460800000
4,1986-08-01 00:00:00,0.098958,0.106771,0.094618,0.098958,0.061350,266256000


In [21]:
df_microsoft_quarterly.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-01 00:00:00,0.095486,0.123264,0.092882,0.121528,0.075343,880588800
1,1986-06-01 00:00:00,0.121528,0.121528,0.090278,0.098958,0.061350,1020844800
2,1986-09-01 00:00:00,0.098958,0.177951,0.091146,0.172743,0.107094,2904796800
3,1986-12-01 00:00:00,0.172743,0.276042,0.157118,0.266493,0.165216,4517395200
4,1987-03-01 00:00:00,0.266493,0.445313,0.255208,0.400174,0.248093,6093964800


Podstawowe informacje o wczytanych danych

In [23]:
df_microsoft_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       9524 non-null   object 
 1   Open       9524 non-null   float64
 2   High       9524 non-null   float64
 3   Low        9524 non-null   float64
 4   Close      9524 non-null   float64
 5   Adj Close  9524 non-null   float64
 6   Volume     9524 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 521.0+ KB


In [24]:
df_microsoft_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1973 entries, 0 to 1972
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1973 non-null   object 
 1   Open       1973 non-null   float64
 2   High       1973 non-null   float64
 3   Low        1973 non-null   float64
 4   Close      1973 non-null   float64
 5   Adj Close  1973 non-null   float64
 6   Volume     1973 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 108.0+ KB


In [25]:
df_microsoft_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       453 non-null    object 
 1   Open       453 non-null    float64
 2   High       453 non-null    float64
 3   Low        453 non-null    float64
 4   Close      453 non-null    float64
 5   Adj Close  453 non-null    float64
 6   Volume     453 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 24.9+ KB


In [26]:
df_microsoft_quarterly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       152 non-null    object 
 1   Open       152 non-null    float64
 2   High       152 non-null    float64
 3   Low        152 non-null    float64
 4   Close      152 non-null    float64
 5   Adj Close  152 non-null    float64
 6   Volume     152 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 8.4+ KB


Widać, że dane zostały wprowadzone prawidłowo. Liczba rekordów jest zgodna z oczekiwaniami. Typy danych są także zgodne z oczekiwaniami oprócz kolumny Date, w któej data jest typu object.

Czyszczenie danych

In [27]:
# ogólne informacje o wczytanych danych

df_microsoft_daily.describe()

,Open,High,Low,Close,Adj Close,Volume
count,9524.000000,9524.000000,9524.000000,9524.000000,9524.000000,9.524000e+03
mean,52.820706,53.383050,52.249194,52.836106,47.328041,5.733973e+07
std,78.772359,79.565262,77.951983,78.800088,78.820530,3.815727e+07
min,0.088542,0.092014,0.088542,0.090278,0.055969,2.304000e+06
25%,5.523438,5.615967,5.446045,5.529297,3.427964,3.300222e+07
50%,27.200001,27.469999,26.969999,27.219999,19.103064,5.049085e+07
75%,45.437500,46.060626,44.870001,45.471562,35.911226,7.123635e+07
max,383.760010,384.299988,378.160004,382.700012,382.700012,1.031789e+09


In [28]:
df_microsoft_weekly.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1973.000000,1973.000000,1973.000000,1973.000000,1973.000000,1.973000e+03
mean,52.838551,54.326388,51.543440,53.074646,47.549383,2.767949e+08
std,78.762880,80.933594,76.939752,79.182533,79.204835,1.443870e+08
min,0.088542,0.096354,0.088542,0.092882,0.057583,2.272320e+07
25%,5.539063,5.781250,5.335938,5.609375,3.477609,1.702224e+08
50%,27.205000,27.840000,26.590000,27.270000,19.065594,2.610543e+08
75%,45.430000,46.900002,44.187500,45.656250,36.053196,3.485209e+08
max,376.779999,384.299988,373.500000,377.429993,377.429993,1.339949e+09


In [29]:
df_microsoft_monthly.describe()

,Open,High,Low,Close,Adj Close,Volume
count,453.000000,453.000000,453.000000,453.000000,453.000000,4.530000e+02
mean,52.745877,56.126207,49.926336,53.503895,47.945798,1.201455e+09
std,78.739213,83.521793,74.587995,80.006665,80.011090,5.048074e+08
min,0.095486,0.106771,0.090278,0.098090,0.060812,2.662560e+08
25%,5.656250,6.117188,5.210938,5.656250,3.506670,8.004316e+08
50%,27.250000,28.549999,25.719999,27.299999,19.015331,1.219400e+09
75%,45.093750,47.812500,42.187500,45.281250,35.626637,1.472519e+09
max,376.760010,384.299988,362.899994,378.910004,378.142517,3.567226e+09


In [30]:
df_microsoft_quarterly.describe()

,Open,High,Low,Close,Adj Close,Volume
count,152.000000,152.000000,152.000000,152.000000,152.000000,1.520000e+02
mean,53.234635,59.777967,48.725704,55.636722,50.122207,3.580653e+09
std,80.768072,89.806715,73.835863,84.589031,84.679789,1.347780e+09
min,0.095486,0.121528,0.090278,0.098958,0.061350,4.866583e+08
25%,5.412110,6.472657,5.015625,5.697266,3.532099,2.261966e+09
50%,26.965000,29.898125,24.530000,27.035001,19.138899,3.776785e+09
75%,44.966875,49.667500,40.147499,45.453360,37.992473,4.403807e+09
max,376.760010,384.299988,362.899994,378.910004,378.142517,6.767030e+09
